In [14]:
import pandas as pd
import json
from collections import defaultdict
csv  = pd.read_csv("../raw/domain expert knowledge - Sheet1.csv")
with open('../programs/programs.json', 'r') as f:
    programs = json.load(f)

In [15]:
csv

,School name,Programms,Intake,Unnamed: 3,Unnamed: 4,Unnamed: 5,@
0,emlyon Business School,Mastère Spécialisés® (MS) Entrepreneuriat & Ma...,Fall,NaN,NaN,NaN,NaN
1,emlyon Business School,Mastère Spécialisés® (MS) Stratégie & Développ...,Fall,NaN,NaN,NaN,NaN
2,emlyon Business School,Mastère Spécialisés® (MS) Juriste Manager Inte...,Fall,NaN,NaN,NaN,NaN
3,emlyon Business School,Mastère Spécialisés® (MS) Transformation Digit...,Fall,NaN,NaN,NaN,NaN
4,Montpellier Business School,Master in Management (Master Grande École),Fall,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
57,EXCELIA Group,MSc Management Intégré des Achats et de la Sup...,Fall,NaN,French,NaN,NaN
58,EXCELIA Group,MSc Management du Tourisme et de l'Événementiel,Fall,NaN,French,NaN,NaN
59,EXCELIA Group,MSc Gestion des Services Immobiliers,Fall,NaN,French,NaN,NaN
60,EM Strasbourg Business School,Programme Grande École,Fall,Year 4,French,Parcours immersif: \ntrès peu de places dispon...,NaN


In [16]:
# change the unnamed column 3 to 'year'
csv.rename(columns={csv.columns[3]: 'year'}, inplace=True)
csv.rename(columns={csv.columns[4]: 'language'}, inplace=True)
csv.fillna('None', inplace=True)

/var/folders/cy/dc_8pgb57s94_82bd8_bjxk00000gn/T/ipykernel_1287/907912653.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  csv.fillna('None', inplace=True)


In [17]:
csv['year'].unique()

array(['None', 'Year 2 only', 'Year 1', 'Year 4'], dtype=object)

In [18]:
#remove only from year 2 only in year column
csv['year'] = csv.apply(lambda row: row['year'] if row['year'] != 'Year 2 only' else 'Year 2', axis=1)

In [19]:
csv['year'].unique()

array(['None', 'Year 2', 'Year 1', 'Year 4'], dtype=object)

In [20]:
schools = csv['School name']
primos_arrivant_programs = csv['Programms']

In [21]:
programs_schools = defaultdict(dict)
for program, school, year in zip(primos_arrivant_programs, schools, csv['year']):
    programs_schools[program] = {'school': school, 'year': year}

In [22]:
programs_schools

defaultdict(dict,
            {'Mastère Spécialisés® (MS) Entrepreneuriat & Management de l’Innovation': {'school': 'emlyon Business School',
              'year': 'None'},
             'Mastère Spécialisés® (MS) Stratégie & Développement d’Affaires Internationales': {'school': 'emlyon Business School',
              'year': 'None'},
             'Mastère Spécialisés® (MS) Juriste Manager International ': {'school': 'emlyon Business School',
              'year': 'None'},
             'Mastère Spécialisés® (MS) Transformation Digitale, Marketing & Stratégie': {'school': 'emlyon Business School',
              'year': 'None'},
             'Master in Management (Master Grande École) ': {'school': 'EM Strasbourg Business School',
              'year': 'Year 1'},
             'Mastère Spécialisé® Gestion Patrimoniale et Immobilière': {'school': 'KEDGE Business School',
              'year': 'None'},
             'Mastère Spécialisé® Innovation et Santé': {'school': 'KEDGE Business School'

## Map primos_arrivant values based on CSV data

For each program in the JSON:
1. Set program-level `primos_arrivant` to `True` if the school+program combination exists in CSV
2. Set year-level `primos_arrivant` to `True` for the specific year mentioned in CSV
3. Set all other `primos_arrivant` values to `False`

In [25]:
# Create a mapping of (school, program) -> year for quick lookup
school_program_year_map = {}
for _, row in csv.iterrows():
    key = (row['School name'].strip(), row['Programms'].strip())
    school_program_year_map[key] = row['year']

print(f"Total CSV entries: {len(school_program_year_map)}")
print(f"\nSample mappings:")
for i, (key, year) in enumerate(list(school_program_year_map.items())[:3]):
    print(f"  {key[0]} - {key[1]} -> {year}")

Total CSV entries: 60

Sample mappings:
  emlyon Business School - Mastère Spécialisés® (MS) Entrepreneuriat & Management de l’Innovation -> None
  emlyon Business School - Mastère Spécialisés® (MS) Stratégie & Développement d’Affaires Internationales -> None
  emlyon Business School - Mastère Spécialisés® (MS) Juriste Manager International -> None


In [26]:
# Function to normalize year string for matching
def normalize_year(year_str):
    """Convert 'Year 1', 'Year 2', etc. to 'year_1', 'year_2' format"""
    if year_str == 'None':
        return None
    # Extract the year number
    if 'Year' in year_str:
        year_num = year_str.split()[1]
        return f'year_{year_num}'
    return None

# Test the function
print("Testing year normalization:")
print(f"  'Year 1' -> {normalize_year('Year 1')}")
print(f"  'Year 2' -> {normalize_year('Year 2')}")
print(f"  'Year 4' -> {normalize_year('Year 4')}")
print(f"  'None' -> {normalize_year('None')}")

Testing year normalization:
  'Year 1' -> year_1
  'Year 2' -> year_2
  'Year 4' -> year_4
  'None' -> None


In [27]:
# Update programs with primos_arrivant values
updated_count = 0
matched_programs = []
unmatched_csv_entries = []

for program in programs:
    school_name = program['school'].strip()
    program_name = program['program'].strip()
    
    # Check if this school+program combination exists in CSV
    key = (school_name, program_name)
    
    if key in school_program_year_map:
        # Program found in CSV - set program-level primos_arrivant to True
        program['primos_arrivant'] = True
        csv_year = school_program_year_map[key]
        normalized_year = normalize_year(csv_year)
        
        matched_programs.append({
            'school': school_name,
            'program': program_name,
            'csv_year': csv_year,
            'normalized_year': normalized_year
        })
        
        # Update year-level primos_arrivant
        if 'year_details' in program and program['year_details']:
            for year_key, intakes in program['year_details'].items():
                for intake in intakes:
                    # Set to True only if this is the specified year, otherwise False
                    if normalized_year and year_key == normalized_year:
                        intake['primos_arrivant'] = True
                    elif normalized_year is None:
                        # If no specific year mentioned, set all years to True
                        intake['primos_arrivant'] = True
                    else:
                        intake['primos_arrivant'] = False
        
        updated_count += 1
    else:
        # Program not found in CSV - set all primos_arrivant to False
        program['primos_arrivant'] = False
        
        if 'year_details' in program and program['year_details']:
            for year_key, intakes in program['year_details'].items():
                for intake in intakes:
                    intake['primos_arrivant'] = False

print(f"✅ Updated {updated_count} programs with primos_arrivant = True")
print(f"📊 Total programs processed: {len(programs)}")
print(f"🔍 Programs not in CSV (set to False): {len(programs) - updated_count}")

✅ Updated 60 programs with primos_arrivant = True
📊 Total programs processed: 1410
🔍 Programs not in CSV (set to False): 1350


In [28]:
# Display some matched programs for verification
print("Sample matched programs:")
for i, match in enumerate(matched_programs[:5]):
    print(f"\n{i+1}. {match['school']}")
    print(f"   Program: {match['program']}")
    print(f"   CSV Year: {match['csv_year']} -> {match['normalized_year']}")

Sample matched programs:

1. emlyon Business School
   Program: Mastère Spécialisés® (MS) Entrepreneuriat & Management de l’Innovation
   CSV Year: None -> None

2. emlyon Business School
   Program: Mastère Spécialisés® (MS) Stratégie & Développement d’Affaires Internationales
   CSV Year: None -> None

3. emlyon Business School
   Program: Mastère Spécialisés® (MS) Juriste Manager International
   CSV Year: None -> None

4. emlyon Business School
   Program: Mastère Spécialisés® (MS) Transformation Digitale, Marketing & Stratégie
   CSV Year: None -> None

5. Montpellier Business School
   Program: Master in Management (Master Grande École)
   CSV Year: None -> None


In [29]:
# Verify one updated program (let's check EPSI - Expert en cybersécurité)
for program in programs:
    if "Expert en cybersécurité" in program['program'] and "EPSI" in program['school']:
        print("Verification - EPSI Expert en cybersécurité:")
        print(f"  Program-level primos_arrivant: {program['primos_arrivant']}")
        print(f"  Year details:")
        if program['year_details']:
            for year, intakes in program['year_details'].items():
                print(f"    {year}:")
                for intake in intakes:
                    print(f"      Intake: {intake.get('program_intake', 'N/A')}, primos_arrivant: {intake.get('primos_arrivant', 'N/A')}")
        break

Verification - EPSI Expert en cybersécurité:
  Program-level primos_arrivant: False
  Year details:
    year_1:
      Intake: Fall, primos_arrivant: False
    year_2:
      Intake: Fall, primos_arrivant: False


In [30]:
# Check for CSV entries that didn't match any program
csv_entries = set(school_program_year_map.keys())
json_entries = set((p['school'].strip(), p['program'].strip()) for p in programs)

unmatched_in_csv = csv_entries - json_entries

print(f"\n⚠️  CSV entries not found in JSON ({len(unmatched_in_csv)}):")
for school, program in list(unmatched_in_csv)[:10]:
    year = school_program_year_map[(school, program)]
    print(f"  - {school}: {program} (Year: {year})")
    
if len(unmatched_in_csv) > 10:
    print(f"  ... and {len(unmatched_in_csv) - 10} more")


⚠️  CSV entries not found in JSON (0):


## Save updated programs to JSON file

In [31]:
# Save the updated programs back to JSON
output_path = '../programs/programs.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(programs, f, ensure_ascii=False, indent=4)

print(f"✅ Successfully saved updated programs to {output_path}")
print(f"📊 Summary:")
print(f"   - Total programs: {len(programs)}")
print(f"   - Programs with primos_arrivant=True: {updated_count}")
print(f"   - Programs with primos_arrivant=False: {len(programs) - updated_count}")

✅ Successfully saved updated programs to ../programs/programs.json
📊 Summary:
   - Total programs: 1410
   - Programs with primos_arrivant=True: 60
   - Programs with primos_arrivant=False: 1350


In [32]:
# Let's verify a program with a specific year mentioned (e.g., ICN Business School - MSc Audit and Compliance - Year 2 only)
for program in programs:
    if "ICN Business School" in program['school'] and "MSc Audit and Compliance" in program['program']:
        print("Verification - ICN Business School - MSc Audit and Compliance (should have Year 2 only):")
        print(f"  Program-level primos_arrivant: {program['primos_arrivant']}")
        print(f"  Year details:")
        if program['year_details']:
            for year, intakes in program['year_details'].items():
                print(f"    {year}:")
                for intake in intakes:
                    print(f"      Intake: {intake.get('program_intake', 'N/A')}, primos_arrivant: {intake.get('primos_arrivant', 'N/A')}")
        break

Verification - ICN Business School - MSc Audit and Compliance (should have Year 2 only):
  Program-level primos_arrivant: True
  Year details:
    year_2:
      Intake: Fall, primos_arrivant: True
      Intake: Fall, primos_arrivant: True
    year_1:
      Intake: None, primos_arrivant: False


In [33]:
# Let's verify Toulouse Business School - Master in Management (Year 1)
for program in programs:
    if "Toulouse Business School" in program['school'] and "Master in Management (Master Grande Ecole)" in program['program']:
        print("Verification - Toulouse Business School - Master in Management (should have Year 1):")
        print(f"  Program-level primos_arrivant: {program['primos_arrivant']}")
        print(f"  Year details:")
        if program['year_details']:
            for year, intakes in program['year_details'].items():
                print(f"    {year}:")
                for idx, intake in enumerate(intakes):
                    if idx < 2:  # Show first 2 intakes only
                        print(f"      Intake: {intake.get('program_intake', 'N/A')}, primos_arrivant: {intake.get('primos_arrivant', 'N/A')}, language: {intake.get('intake_language', 'N/A')}")
        break

Verification - Toulouse Business School - Master in Management (should have Year 1):
  Program-level primos_arrivant: True
  Year details:
    year_1:
      Intake: Fall, primos_arrivant: True, language: French
      Intake: Fall, primos_arrivant: True, language: English
    year_2:
      Intake: None, primos_arrivant: False, language: French
      Intake: None, primos_arrivant: False, language: English


## ✅ Summary

The `primos_arrivant` mapping has been successfully completed:

1. **60 programs** from the CSV have been marked with `primos_arrivant = True` at the program level
2. For programs with **specific years mentioned** in the CSV (e.g., "Year 1", "Year 2"):
   - Only the specified year's intakes have `primos_arrivant = True`
   - All other years have `primos_arrivant = False`
3. For programs with **no specific year mentioned** (None):
   - All years' intakes have `primos_arrivant = True`
4. **1,350 programs** not in the CSV have all `primos_arrivant` fields set to `False`

The updated data has been saved to `../programs/programs.json`

## 📝 Integration Complete

The primos_arrivant mapping logic has been extracted into a reusable script and integrated into the data pipeline:

### Created Files:
- **`data/operations/primos_arrivant_mapper.py`**: Standalone script with all the mapping logic

### Updated Files:
- **`data/operations/full_data_pipeline.py`**: Added Step 11 to run primos_arrivant mapping

### How to Use:

**Standalone:**
```bash
cd data/operations
python primos_arrivant_mapper.py
```

**As part of the full pipeline:**
```bash
cd data/operations
python full_data_pipeline.py
```

The script will automatically:
1. Load the CSV from `data/raw/domain expert knowledge - Sheet1.csv`
2. Load the programs from `data/programs/programs.json`
3. Apply the primos_arrivant mapping
4. Save the updated programs back to `data/programs/programs.json`

In [34]:
# Verify the script was created successfully
import os
script_path = "primos_arrivant_mapper.py"
if os.path.exists(script_path):
    print(f"✅ Script created: {script_path}")
    with open(script_path, 'r') as f:
        lines = f.readlines()
    print(f"   Lines of code: {len(lines)}")
    print(f"   Functions: normalize_year, load_primos_arrivant_csv, apply_primos_arrivant_mapping, map_primos_arrivant")
else:
    print(f"❌ Script not found: {script_path}")

✅ Script created: primos_arrivant_mapper.py
   Lines of code: 219
   Functions: normalize_year, load_primos_arrivant_csv, apply_primos_arrivant_mapping, map_primos_arrivant


## 🎯 What the Script Does

The `primos_arrivant_mapper.py` script:

1. **Loads Data**:
   - CSV file with school/program/year information
   - Existing programs JSON

2. **Preprocesses CSV**:
   - Renames columns
   - Handles "Year 2 only" → "Year 2"
   - Creates lookup mapping

3. **Applies Mapping Logic**:
   - For each program in JSON:
     - If found in CSV → `primos_arrivant = True` (program level)
     - If specific year mentioned → only that year's intakes get `True`
     - If no year mentioned → all years' intakes get `True`
     - If not in CSV → all `primos_arrivant = False`

4. **Saves Results**:
   - Updates programs JSON with new values
   - Provides summary statistics

5. **Path Handling**:
   - Works from both root directory and operations directory
   - Auto-detects and adjusts paths accordingly